In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from pystacknet.pystacknet import StackNetRegressor
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from fastFM import als, mcmc, sgd

from sklearn.metrics import mean_squared_error, mean_absolute_error

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP

import types

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
mytrial = list(pd.read_pickle('../trial/pystacknet.pkl').T.to_dict().values())
df_trial = pd.DataFrame(mytrial)
len(mytrial)

2

In [7]:
df_trial[['n_columns','train_mae','remark']]

,n_columns,train_mae,remark
0,30,1.721495,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
1,30,1.664118,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->CatBoostRegressor->


In [8]:
df_trial_catboost = pd.read_pickle('../trial/catboost.pkl')
df_trial_catboost.loc[452:452][['datetime','remark','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
452,2019-05-16 06:46:11.662876,tune 437,30,1.853478,0.000005,1.900535,0.000072,0.047056


In [9]:
df_trial_xgbm = pd.read_pickle('../trial/xgbm.pkl')
df_trial_xgbm.loc[1172:1172][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
1172,2019-05-16 09:18:54.750120,tune 1099,25,1.924779,0.000016,1.96556,0.000066,0.040782


In [10]:
df_trial_lgbm = pd.read_pickle('../trial/lgbm.pkl')
df_trial_lgbm.loc[2156:2156][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
2156,2019-05-16 22:30:28.865774,tune 2025,25,1.911171,0.00002,1.952897,0.000129,0.041726


In [35]:
df_trial_lgbm.loc[2156]['param']

{'columns': ['q25_roll_std_100',
  'abs_q01_4',
  'q05_roll_std_1000',
  'abs_q25_5',
  'spkt_welch_densitycoeff_2',
  "number_peaks{'n': 10}",
  'spkt_welch_density__coeff_3',
  'iqr_6',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'abs_q75_7',
  '5000clearance_factor_quantile25',
  '5000smoothness_std_',
  'q05_roll_std_100',
  'spkt_welch_density__coeff_25',
  'abs_q75_6',
  'abs_max_1',
  'median__roll_std',
  'q05_roll_std_10',
  'abs_q05_7',
  "autocorrelation{'lag': 5}",
  'abs_max_7',
  'FFT_Mag_95q0',
  'abs_max_8',
  '5000smoothness_entropy_',
  'spkt_welch_density__coeff_42'],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'lgb.LGBMRegressor',
  'init': {'learning_rate': 0.19329183957823715,
   'feature_fraction': 0.7363782645537933,
   'bagging_fraction': 0.8585309819625903,
   'min_data_in_leaf': 351,
   'lambda_l1': 95.63411922439124,

In [11]:
df_trial_randomforest = pd.read_pickle('../trial/randomforest.pkl')
df_trial_randomforest.loc[297:297][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
297,2019-05-16 12:50:31.611238,tune 239,80,2.080347,0.000003,2.086208,0.000063,0.00586


In [12]:
df_trial_extratrees = pd.read_pickle('../trial/extratrees.pkl')
df_trial_extratrees.loc[459:459][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
459,2019-05-16 11:30:08.432873,tune 427,70,2.109147,0.000011,2.110347,0.000025,0.0012


In [13]:
df_trial_gradientboosting = pd.read_pickle('../trial/gradientboosting.pkl')
df_trial_gradientboosting.loc[306:306][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
306,2019-05-16 14:16:23.815368,tune 220,70,1.914761,0.000019,1.951837,0.000144,0.037076


In [14]:
df_trial_knn = pd.read_pickle('../trial/knn.pkl')
df_trial_knn.loc[17:17][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
17,2019-05-16 23:11:48.201772,None,7,2.026411,0.002031,2.081674,0.013139,0.055264


In [15]:
df_trial_svr = pd.read_pickle('../trial/svr.pkl')
df_trial_svr.loc[7:7][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
7,2019-05-16 23:57:06.120463,None,7,2.050366,0.003073,2.092532,0.01549,0.042166


In [16]:
df_trial_fm = pd.read_pickle('../trial/fm.pkl')
df_trial_fm.loc[0:0][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-15 05:53:03.716094,None,165,1.967943,0.000024,2.084029,0.000882,0.116085


In [17]:
df_trial_lasso = pd.read_pickle('../trial/lasso.pkl')
df_trial_lasso.loc[8:8][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
8,2019-05-16 11:54:52.999400,None,165,2.106014,0.002088,2.156098,0.008084,0.050085


In [18]:
df_trial_ridge = pd.read_pickle('../trial/ridge.pkl')
df_trial_ridge.loc[15:15][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
15,2019-05-16 11:27:58.322760,None,165,2.14516,0.002662,2.180493,0.012586,0.035332


In [19]:
def stacknet_params(models):
    model_names = []
    for lv_idx,models_current_lv  in enumerate(models):
        model_names.append([(model.__class__.__name__,model.get_params())   for model in models_current_lv])
    return model_names

def stacknet_remark(models):
    remark = ''
    for lv_idx,models_current_lv  in enumerate(models):
        remark += '|'.join([model.__class__.__name__ for model in models_current_lv])
        remark += '->'
        
    return remark

def process(df_train, df_test, columns, models, trial , remark=None, restacking=True, use_retraining=True, folds=3):
    
    X_train,y_train = df_train[columns].values, df_train['y'].values
    X_test = df_test[columns].values
    scaler = StandardScaler()
    scaler.fit(np.concatenate([X_train, X_test], axis=0))
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    stacknet = StackNetRegressor(models, metric="mae", folds=folds, restacking=restacking, use_retraining=use_retraining, random_state=12345, n_jobs=1, verbose=1)
    stacknet.fit(X_train,y_train)
    y_train_pred = stacknet.predict(X_train)
    y_test_pred = stacknet.predict(X_test)
    
    trial.append({'n_columns':len(columns), 'columns':columns, 'remark':stacknet_remark(models), 'models':stacknet_params(models), 'y_train_pred':y_train_pred, 'y_test_pred':y_test_pred, 'train_mae':mean_absolute_error(y_train, y_train_pred)})
    return stacknet


def process2(df_train, df_test, columns, models, splits):
    
    trial = []
    X_train,y_train = df_train[columns].values, df_train['y'].values
    X_test = df_test[columns].values
    scaler = StandardScaler()
    scaler.fit(np.concatenate([X_train, X_test], axis=0))
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    for fold_n, (train_index, valid_index) in enumerate(splits):
        for m in models:
            m.fit(X_train[train_index], y_train[train_index])
            y_train_pred = m.predict(X_train[train_index])
            y_valid_pred = m.predict(X_train[valid_index])
            metric_train = mean_absolute_error(y_train[train_index], y_train_pred)
            metric_valid = mean_absolute_error(y_train[valid_index], y_valid_pred)
            trial.append({'algorithm':m.__class__.__name__, 'fold_n':fold_n, 'n_columns':len(columns), 'metric_train':metric_train, 'metric_valid':metric_valid, 'y_train_pred':y_train_pred, 'y_valid_pred':y_valid_pred, 'columns':columns})
            
            
            
            
    return pd.DataFrame(trial)

In [22]:
columns_ = df_trial_catboost.loc[452]['param']['columns']+df_trial_xgbm.loc[1172]['param']['columns']+df_trial_lgbm.loc[2156]['param']['columns']+df_trial_randomforest.loc[297]['param']['columns']+df_trial_extratrees.loc[459]['param']['columns']+df_trial_gradientboosting.loc[306]['param']['columns']

unique_columns = list(set(columns_))
common_columns = []
main_columns = []
for col in unique_columns:
    if columns_.count(col)==6:
        common_columns.append(col)
    if columns_.count(col)>int(6*.8):
        main_columns.append(col)
len(unique_columns), len(common_columns),len(main_columns)
catboost_columns = df_trial_catboost.loc[452]['param']['columns'].copy()

In [38]:
trial_lst = [
    df_trial_catboost.loc[452], df_trial_xgbm.loc[1172], df_trial_lgbm.loc[2156],df_trial_gradientboosting.loc[306],
             df_trial_randomforest.loc[297],df_trial_extratrees.loc[459],
#             df_trial_knn.loc[17], df_trial_svr.loc[7],df_trial_fm.loc[313],
            df_trial_lasso.loc[8], df_trial_ridge.loc[15],
            ]
name_lst = [
    'catboost452', 'xgbm1172', 'lgbm2156',
    'gradientboosting306','randomforest297','extratrees459',
#     'knn17','svr7', 'fm313',
    'lasso8','ridge15',

]

In [39]:
models=[ 
######## First level ########
    [
        RandomForestRegressor (**df_trial_randomforest.loc[297]['param']['algorithm']['init']),
        ExtraTreesRegressor (**df_trial_extratrees.loc[459]['param']['algorithm']['init']),
        GradientBoostingRegressor(**df_trial_gradientboosting.loc[306]['param']['algorithm']['init']),
        cb.CatBoostRegressor(**df_trial_catboost.loc[452]['param']['algorithm']['init']),
        lgb.LGBMRegressor(**df_trial_lgbm.loc[2156]['param']['algorithm']['init']),
        xgb.XGBRegressor(**df_trial_xgbm.loc[1172]['param']['algorithm']['init']),
#         KNeighborsRegressor(**df_trial_knn.loc[17]['param']['algorithm']['init']),
#         SVR(**df_trial_svr.loc[7]['param']['algorithm']['init']),
        Lasso(**df_trial_lasso.loc[8]['param']['algorithm']['init']),
        Ridge(**df_trial_ridge.loc[15]['param']['algorithm']['init']),
#         als.FMRegression(**df_trial_fm.loc[9]['param']['algorithm']['init']),
     ],
######## Second level ########
    [
        cb.CatBoostRegressor(**df_trial_catboost.loc[452]['param']['algorithm']['init']),
#         xgb.XGBRegressor(**df_trial_xgbm.loc[865]['param']['algorithm']['init']),
#         RandomForestRegressor (**df_trial_randomforest.loc[122]['param']['algorithm']['init']),
#         SVR(**df_trial_svr.loc[0]['param']['algorithm']['init']),
#         Ridge(**df_trial_ridge.loc[3]['param']['algorithm']['init']),
    ],
######## third level ########
#     [
#         Ridge(**df_trial_ridge.loc[3]['param']['algorithm']['init'])
#     ]
]

In [48]:
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

kfold_type = 'StratifiedKFold'
n_splits = 3

if kfold_type == 'StratifiedKFold':
    folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    splits = list(folds.split(df_train, df_train['label']))
elif kfold_type == 'GroupKFold':
    folds = GroupKFold(n_splits=3)
    splits = list(folds.split(df_train, groups=df_train['group']))
else:
    folds = KFold(n_splits=3, shuffle=True, random_state=42)
    splits = list(folds.split(df_train))

In [55]:
# mytrial = []

In [84]:
# df_trial2 = process2(df_train, df_test, columns, models[0],splits=splits)

In [50]:
# df_trial2['n_columns'] = df_trial2['columns'].apply(lambda x: len(x))
# df_trial2['metric_diff'] =  df_trial2['metric_valid'] - df_trial2['metric_train']

In [85]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(25,5))
# N_ = splits[0][1].shape[0]
# sns.lineplot(x=np.arange(N_) ,y=df_train['y'].values[splits[0][1]])
# sns.lineplot(x=np.arange(N_) ,y=df_trial2[(df_trial2['algorithm']=='KNeighborsRegressor')&(df_trial2['fold_n']==0)]['y_valid_pred'].values[0])

In [86]:
# df_trial2[['algorithm', 'fold_n', 'n_columns', 'metric_train', 'metric_valid', 'metric_diff']].sort_values(by=['algorithm','fold_n'])

In [87]:
# df_trial2[(df_trial2['fold_n']==0)][['algorithm', 'fold_n', 'n_columns', 'metric_train', 'metric_valid', 'metric_diff']].sort_values(by=['algorithm','fold_n'])

In [71]:
# sns.pairplot(pd.DataFrame(np.array(df_trial2[(df_trial2['fold_n']==0)]['y_valid_pred'].tolist()).T, columns=df_trial2[(df_trial2['fold_n']==0)]['algorithm'].tolist()))

In [49]:
columns = catboost_columns
stacknet = process(df_train, df_test, columns, models, mytrial, folds=splits)

====================== Start of Level 0 ======================
Input Dimensionality 30 at Level 0 
8 models included in Level 0 
0:	learn: 6.6545450	total: 7.46ms	remaining: 6.92s
1:	learn: 6.5675322	total: 16.8ms	remaining: 7.81s
2:	learn: 6.4825868	total: 24.4ms	remaining: 7.53s
3:	learn: 6.3989199	total: 33.4ms	remaining: 7.72s
4:	learn: 6.3168110	total: 40.2ms	remaining: 7.43s
5:	learn: 6.2359546	total: 49.8ms	remaining: 7.67s
6:	learn: 6.1564341	total: 59.9ms	remaining: 7.89s
7:	learn: 6.0788639	total: 69.6ms	remaining: 8.01s
8:	learn: 6.0029087	total: 79.2ms	remaining: 8.1s
9:	learn: 5.9278289	total: 87.7ms	remaining: 8.06s
10:	learn: 5.8540124	total: 96.4ms	remaining: 8.05s
11:	learn: 5.7819184	total: 105ms	remaining: 8.05s
12:	learn: 5.7108970	total: 114ms	remaining: 8.05s
13:	learn: 5.6412626	total: 121ms	remaining: 7.92s
14:	learn: 5.5730997	total: 130ms	remaining: 7.92s
15:	learn: 5.5059586	total: 139ms	remaining: 7.94s
16:	learn: 5.4408678	total: 149ms	remaining: 7.99s
17:	

158:	learn: 2.6260620	total: 1.47s	remaining: 7.13s
159:	learn: 2.6237759	total: 1.48s	remaining: 7.12s
160:	learn: 2.6215408	total: 1.49s	remaining: 7.12s
161:	learn: 2.6191951	total: 1.5s	remaining: 7.11s
162:	learn: 2.6173819	total: 1.51s	remaining: 7.1s
163:	learn: 2.6151664	total: 1.52s	remaining: 7.09s
164:	learn: 2.6128110	total: 1.53s	remaining: 7.09s
165:	learn: 2.6106920	total: 1.54s	remaining: 7.08s
166:	learn: 2.6086046	total: 1.55s	remaining: 7.08s
167:	learn: 2.6065420	total: 1.56s	remaining: 7.07s
168:	learn: 2.6042248	total: 1.57s	remaining: 7.06s
169:	learn: 2.6025081	total: 1.58s	remaining: 7.05s
170:	learn: 2.6005354	total: 1.59s	remaining: 7.04s
171:	learn: 2.5988537	total: 1.6s	remaining: 7.03s
172:	learn: 2.5971157	total: 1.61s	remaining: 7.02s
173:	learn: 2.5952967	total: 1.61s	remaining: 7s
174:	learn: 2.5935152	total: 1.62s	remaining: 7s
175:	learn: 2.5916738	total: 1.63s	remaining: 6.99s
176:	learn: 2.5901061	total: 1.64s	remaining: 6.97s
177:	learn: 2.5884037

317:	learn: 2.4841313	total: 2.94s	remaining: 5.65s
318:	learn: 2.4837463	total: 2.95s	remaining: 5.65s
319:	learn: 2.4835853	total: 2.96s	remaining: 5.64s
320:	learn: 2.4833101	total: 2.97s	remaining: 5.63s
321:	learn: 2.4828301	total: 2.98s	remaining: 5.62s
322:	learn: 2.4824545	total: 2.99s	remaining: 5.62s
323:	learn: 2.4820079	total: 3s	remaining: 5.61s
324:	learn: 2.4814928	total: 3.01s	remaining: 5.6s
325:	learn: 2.4809600	total: 3.02s	remaining: 5.59s
326:	learn: 2.4803565	total: 3.03s	remaining: 5.58s
327:	learn: 2.4799324	total: 3.04s	remaining: 5.57s
328:	learn: 2.4798052	total: 3.05s	remaining: 5.57s
329:	learn: 2.4795189	total: 3.06s	remaining: 5.55s
330:	learn: 2.4792098	total: 3.07s	remaining: 5.54s
331:	learn: 2.4786585	total: 3.08s	remaining: 5.54s
332:	learn: 2.4782786	total: 3.09s	remaining: 5.53s
333:	learn: 2.4776830	total: 3.1s	remaining: 5.52s
334:	learn: 2.4774017	total: 3.11s	remaining: 5.51s
335:	learn: 2.4770825	total: 3.12s	remaining: 5.5s
336:	learn: 2.4766

476:	learn: 2.4314875	total: 4.42s	remaining: 4.19s
477:	learn: 2.4312402	total: 4.43s	remaining: 4.18s
478:	learn: 2.4309780	total: 4.43s	remaining: 4.16s
479:	learn: 2.4306808	total: 4.44s	remaining: 4.15s
480:	learn: 2.4304354	total: 4.45s	remaining: 4.14s
481:	learn: 2.4302161	total: 4.46s	remaining: 4.13s
482:	learn: 2.4299556	total: 4.47s	remaining: 4.13s
483:	learn: 2.4296914	total: 4.48s	remaining: 4.12s
484:	learn: 2.4293229	total: 4.49s	remaining: 4.11s
485:	learn: 2.4291710	total: 4.5s	remaining: 4.1s
486:	learn: 2.4288869	total: 4.5s	remaining: 4.09s
487:	learn: 2.4287272	total: 4.51s	remaining: 4.08s
488:	learn: 2.4285831	total: 4.53s	remaining: 4.07s
489:	learn: 2.4283576	total: 4.53s	remaining: 4.06s
490:	learn: 2.4281321	total: 4.54s	remaining: 4.05s
491:	learn: 2.4278821	total: 4.55s	remaining: 4.04s
492:	learn: 2.4275056	total: 4.56s	remaining: 4.04s
493:	learn: 2.4273876	total: 4.57s	remaining: 4.03s
494:	learn: 2.4272055	total: 4.58s	remaining: 4.02s
495:	learn: 2.4

635:	learn: 2.3961246	total: 5.83s	remaining: 2.69s
636:	learn: 2.3959024	total: 5.84s	remaining: 2.68s
637:	learn: 2.3956959	total: 5.85s	remaining: 2.67s
638:	learn: 2.3954388	total: 5.86s	remaining: 2.66s
639:	learn: 2.3951871	total: 5.87s	remaining: 2.65s
640:	learn: 2.3949619	total: 5.88s	remaining: 2.64s
641:	learn: 2.3947254	total: 5.88s	remaining: 2.63s
642:	learn: 2.3945101	total: 5.89s	remaining: 2.62s
643:	learn: 2.3943606	total: 5.9s	remaining: 2.61s
644:	learn: 2.3941251	total: 5.91s	remaining: 2.6s
645:	learn: 2.3939662	total: 5.92s	remaining: 2.59s
646:	learn: 2.3937276	total: 5.92s	remaining: 2.58s
647:	learn: 2.3936520	total: 5.93s	remaining: 2.57s
648:	learn: 2.3934096	total: 5.94s	remaining: 2.56s
649:	learn: 2.3932876	total: 5.95s	remaining: 2.55s
650:	learn: 2.3929952	total: 5.96s	remaining: 2.54s
651:	learn: 2.3927220	total: 5.97s	remaining: 2.54s
652:	learn: 2.3925814	total: 5.98s	remaining: 2.53s
653:	learn: 2.3922839	total: 5.99s	remaining: 2.52s
654:	learn: 2.

...

150:	learn: 2.4864414	total: 1.88s	remaining: 9.69s
151:	learn: 2.4839188	total: 1.89s	remaining: 9.68s
152:	learn: 2.4815040	total: 1.9s	remaining: 9.65s
153:	learn: 2.4792578	total: 1.92s	remaining: 9.64s
154:	learn: 2.4768297	total: 1.93s	remaining: 9.63s
155:	learn: 2.4744947	total: 1.94s	remaining: 9.62s
156:	learn: 2.4722413	total: 1.96s	remaining: 9.61s
157:	learn: 2.4702575	total: 1.97s	remaining: 9.6s
158:	learn: 2.4681865	total: 1.98s	remaining: 9.6s
159:	learn: 2.4660536	total: 1.99s	remaining: 9.59s
160:	learn: 2.4641664	total: 2.01s	remaining: 9.58s
161:	learn: 2.4623528	total: 2.02s	remaining: 9.57s
162:	learn: 2.4605477	total: 2.04s	remaining: 9.56s
163:	learn: 2.4587161	total: 2.05s	remaining: 9.56s
164:	learn: 2.4569367	total: 2.06s	remaining: 9.53s
165:	learn: 2.4550936	total: 2.07s	remaining: 9.52s
166:	learn: 2.4534612	total: 2.08s	remaining: 9.51s
167:	learn: 2.4517981	total: 2.1s	remaining: 9.49s
168:	learn: 2.4501443	total: 2.11s	remaining: 9.48s
169:	learn: 2.44

309:	learn: 2.3773026	total: 3.85s	remaining: 7.68s
310:	learn: 2.3771416	total: 3.86s	remaining: 7.67s
311:	learn: 2.3769224	total: 3.87s	remaining: 7.66s
312:	learn: 2.3767769	total: 3.88s	remaining: 7.65s
313:	learn: 2.3766231	total: 3.9s	remaining: 7.63s
314:	learn: 2.3763581	total: 3.91s	remaining: 7.62s
315:	learn: 2.3762220	total: 3.92s	remaining: 7.61s
316:	learn: 2.3759357	total: 3.94s	remaining: 7.6s
317:	learn: 2.3756772	total: 3.94s	remaining: 7.58s
318:	learn: 2.3754804	total: 3.96s	remaining: 7.57s
319:	learn: 2.3753306	total: 3.97s	remaining: 7.55s
320:	learn: 2.3751267	total: 3.98s	remaining: 7.54s
321:	learn: 2.3750111	total: 3.99s	remaining: 7.52s
322:	learn: 2.3748089	total: 4s	remaining: 7.51s
323:	learn: 2.3746009	total: 4.02s	remaining: 7.5s
324:	learn: 2.3744179	total: 4.03s	remaining: 7.49s
325:	learn: 2.3741834	total: 4.04s	remaining: 7.47s
326:	learn: 2.3739878	total: 4.05s	remaining: 7.46s
327:	learn: 2.3737866	total: 4.07s	remaining: 7.45s
328:	learn: 2.3736

468:	learn: 2.3532472	total: 5.81s	remaining: 5.7s
469:	learn: 2.3530826	total: 5.82s	remaining: 5.68s
470:	learn: 2.3529592	total: 5.83s	remaining: 5.67s
471:	learn: 2.3527963	total: 5.84s	remaining: 5.66s
472:	learn: 2.3526477	total: 5.85s	remaining: 5.64s
473:	learn: 2.3525155	total: 5.87s	remaining: 5.63s
474:	learn: 2.3523315	total: 5.88s	remaining: 5.62s
475:	learn: 2.3521316	total: 5.89s	remaining: 5.61s
476:	learn: 2.3519449	total: 5.9s	remaining: 5.59s
477:	learn: 2.3518224	total: 5.92s	remaining: 5.58s
478:	learn: 2.3517333	total: 5.93s	remaining: 5.57s
479:	learn: 2.3516590	total: 5.94s	remaining: 5.55s
480:	learn: 2.3515057	total: 5.95s	remaining: 5.54s
481:	learn: 2.3512761	total: 5.96s	remaining: 5.53s
482:	learn: 2.3511239	total: 5.97s	remaining: 5.52s
483:	learn: 2.3510215	total: 5.99s	remaining: 5.5s
484:	learn: 2.3509616	total: 6s	remaining: 5.49s
485:	learn: 2.3507931	total: 6.01s	remaining: 5.48s
486:	learn: 2.3507087	total: 6.03s	remaining: 5.47s
487:	learn: 2.3505

627:	learn: 2.3341139	total: 7.79s	remaining: 3.73s
628:	learn: 2.3340394	total: 7.8s	remaining: 3.72s
629:	learn: 2.3338194	total: 7.81s	remaining: 3.71s
630:	learn: 2.3337278	total: 7.82s	remaining: 3.69s
631:	learn: 2.3335870	total: 7.83s	remaining: 3.68s
632:	learn: 2.3334791	total: 7.84s	remaining: 3.67s
633:	learn: 2.3333611	total: 7.86s	remaining: 3.65s
634:	learn: 2.3332237	total: 7.87s	remaining: 3.64s
635:	learn: 2.3331038	total: 7.88s	remaining: 3.63s
636:	learn: 2.3329869	total: 7.89s	remaining: 3.62s
637:	learn: 2.3328415	total: 7.91s	remaining: 3.61s
638:	learn: 2.3326872	total: 7.92s	remaining: 3.6s
639:	learn: 2.3325778	total: 7.93s	remaining: 3.58s
640:	learn: 2.3324416	total: 7.95s	remaining: 3.57s
641:	learn: 2.3323354	total: 7.96s	remaining: 3.56s
642:	learn: 2.3322077	total: 7.97s	remaining: 3.54s
643:	learn: 2.3320515	total: 7.98s	remaining: 3.53s
644:	learn: 2.3318596	total: 8s	remaining: 3.52s
645:	learn: 2.3316358	total: 8.01s	remaining: 3.51s
646:	learn: 2.331

786:	learn: 2.3155657	total: 9.75s	remaining: 1.76s
787:	learn: 2.3154757	total: 9.76s	remaining: 1.75s
788:	learn: 2.3153827	total: 9.77s	remaining: 1.73s
789:	learn: 2.3152623	total: 9.79s	remaining: 1.72s
790:	learn: 2.3150990	total: 9.8s	remaining: 1.71s
791:	learn: 2.3149501	total: 9.81s	remaining: 1.7s
792:	learn: 2.3148233	total: 9.82s	remaining: 1.68s
793:	learn: 2.3147440	total: 9.83s	remaining: 1.67s
794:	learn: 2.3146246	total: 9.85s	remaining: 1.66s
795:	learn: 2.3144690	total: 9.86s	remaining: 1.65s
796:	learn: 2.3143741	total: 9.87s	remaining: 1.63s
797:	learn: 2.3141238	total: 9.88s	remaining: 1.62s
798:	learn: 2.3139900	total: 9.89s	remaining: 1.61s
799:	learn: 2.3139061	total: 9.91s	remaining: 1.6s
800:	learn: 2.3138382	total: 9.92s	remaining: 1.58s
801:	learn: 2.3137318	total: 9.93s	remaining: 1.57s
802:	learn: 2.3136428	total: 9.94s	remaining: 1.56s
803:	learn: 2.3135409	total: 9.96s	remaining: 1.55s
804:	learn: 2.3134466	total: 9.97s	remaining: 1.53s
805:	learn: 2.3

In [52]:
df_trial = pd.DataFrame(mytrial)
df_trial[['n_columns','train_mae','remark']]

,n_columns,train_mae,remark
0,30,1.721495,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
1,30,1.664118,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
2,30,1.657385,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
3,30,1.715103,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|Lasso|Ridge->CatBoostRegressor->
4,7,1.890294,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|Lasso|Ridge->CatBoostRegressor->
5,30,1.766095,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|Lasso|Ridge->CatBoostRegressor->
6,30,1.890454,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|Lasso|Ridge->CatBoostRegressor->
7,30,1.766095,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|Lasso|Ridge->CatBoostRegressor->


In [53]:
df_trial.to_pickle('../trial/pystacknet.pkl')

In [54]:
idx = 7
y_test_pred = df_trial.loc[idx]['y_test_pred'].reshape(-1)
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = y_test_pred
df_submit['seg_id'] = df_test['index']
df_submit.to_csv('submission_pystacknet_{}.csv'.format(idx), index=False)

In [98]:
# df_trial = pd.DataFrame(mytrial)
# df_trial[['n_columns','train_mae','remark']]

,n_columns,train_mae,remark
0,30,1.721495,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
1,30,1.664118,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
2,30,1.657385,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
3,30,1.897036,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
4,30,1.897036,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->CatBoostRegressor->
5,30,1.664747,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|KNeighborsRegressor|SVR|Lasso|Ridge->XGBRegressor|RandomForestRegressor|SVR|Ridge->Ridge->
6,30,1.723844,CatBoostRegressor|LGBMRegressor|XGBRegressor->CatBoostRegressor->
7,30,1.720960,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|Lasso|Ridge->CatBoostRegressor->
8,30,1.735455,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|Lasso|Ridge->XGBRegressor->
9,165,1.463634,RandomForestRegressor|ExtraTreesRegressor|GradientBoostingRegressor|CatBoostRegressor|LGBMRegressor|XGBRegressor|Lasso|Ridge->CatBoostRegressor->
